In [7]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential, model_from_json
from tqdm.notebook import tqdm
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [9]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "Completed")
    return image_paths,labels

In [10]:
train = pd.DataFrame()
train['image'], train['label'] =createdataframe(TRAIN_DIR)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [5]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [19]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [10]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [14]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [15]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\jigne\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [20]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

C:\Users\jigne\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [21]:
x_train = train_features/255.0
x_test = test_features/255.0

In [22]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [23]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [24]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [25]:
model = Sequential()

#convolutional layers
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
#fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))

#output layer
model.add(Dense(7,activation='softmax'))


In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [16]:
model.fit(x=x_train, y=y_train, batch_size=128,epochs= 90, validation_data=(x_test,y_test))

Epoch 1/90


226/226 [==============================] - 223s 978ms/step - loss: 1.8232 - accuracy: 0.2433 - val_loss: 1.8143 - val_accuracy: 0.2583
Epoch 2/90
226/226 [==============================] - 223s 989ms/step - loss: 1.8029 - accuracy: 0.2503 - val_loss: 1.7740 - val_accuracy: 0.2581
Epoch 3/90
226/226 [==============================] - 228s 1s/step - loss: 1.7225 - accuracy: 0.2931 - val_loss: 1.6399 - val_accuracy: 0.3432
Epoch 4/90
226/226 [==============================] - 228s 1s/step - loss: 1.5994 - accuracy: 0.3658 - val_loss: 1.4626 - val_accuracy: 0.4319
Epoch 5/90
226/226 [==============================] - 229s 1s/step - loss: 1.4979 - accuracy: 0.4173 - val_loss: 1.3800 - val_accuracy: 0.4735
Epoch 6/90
226/226 [==============================] - 230s 1s/step - loss: 1.4326 - accuracy: 0.4495 - val_loss: 1.3236 - val_accuracy: 0.4969
Epoch 7/90
226/226 [==============================] - 231s 1s/step - loss: 1.3844 - accuracy: 0.4665 - val_loss: 1.2794 - val_accuracy

In [26]:
# Assuming you've trained your model already using the code provided

# Evaluate the model on the test data
loss, accuracy = model.evaluate(x_test, y_test)

# Print the accuracy
print('Accuracy:', accuracy)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [5]:
model_json =  model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

NameError: name 'model' is not defined

In [28]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [29]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [30]:
def ef(image):
    img = load_img(image,grayscale=True)
    feature = np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0

In [32]:
image='images/train/fear/570.jpg'
print("original image is of neutral")
img = ef(image)
pred = model.predict(img)
pred_label= label[pred.argmax()]
print("model prediction is", pred_label)

original image is of neutral
1/1 [==============================] - 0s 151ms/step
model prediction is neutral


In [49]:
%matplotlib inline

In [4]:
image = 'images/train/disgust/100.jpg'
print("original image is disgust")
img = ef(image)
pred = model.predict(img)
pred_label= label[pred.argmax()]
print("model prediction is", pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

original image is disgust


NameError: name 'load_img' is not defined